# LTSM Project
---

- **Notebook:** Julio César Martínez
- **Posición:** Data Scientist
- **Fecha:** 21-Ene-24

# Introducción
---

El análisis de sentimiento se refiere a un aspecto del campo de procesamiento del lenguaje natural y se dedica exclusivamente a comprender opiniones subjetivas o sentimientos agregados de una variedad de fuentes sobre un solo tema.

Puesto en un contexto comercial, el análisis de sentimiento se refiere a herramientas que identifican y extrapolan información de opiniones y luego mejoran las operaciones comerciales. Esto se hace con la ayuda de una variedad de algoritmos que profundizan en el subcontexto de las opiniones e intentan comprender las actitudes con respecto a un producto o cualquier elemento específico.

El análisis de sentimiento tiene que ver con la extracción de opiniones para comprender el razonamiento del público en general, lo que permite a las empresas examinar el posicionamiento del producto. El análisis de sentimiento se utiliza en muchas áreas diferentes:

- Analítica de productos
- Estudios de mercado
- Hiperpersonalización
- Manejo de reputación
- Percepciones en las relaciones públicas
- Orientación precisa de los clientes
- Reseñas de productos
- Comentarios sobre el producto
- Servicio al cliente eficiente

El análisis de sentimiento juega un papel muy importante para ayudar a las empresas a desarrollar productos y servicios más inteligentes que aborden específicamente las necesidades de los clientes.

**MODELO RECURRENTE**

Dicho lo anterior, en este proyecto estarémos realizando un análisis de sentimientos para una página de películas llamada IMBD, para ello, utilizaremos un modelo recurrente LTSM (long short term memory) donde introducimos información a una capa y después volvemos a meter información a la misma.

# Tabla de Contenido
---

1. Cargar los datos
2. Exploración de datos
3. Preprocesamiento de datos
4. Crear Red Neuronal
5. Entrenar y evaluar modelo
6. Conclusiones

# Proyecto LTSM
---

Para este proyecto utilizaremos un dataset de keras y es con el que vamos a trabajar de manera sencilla para demostrar algunos de nuestros conocimientos en este campo de la inteligencia artificial.

## Cargar datos

cargar librerías

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SimpleRNN
#from tensorflow.keras.layers.embeddings import Embeding

cargar datos

In [2]:
# definimos número de palabras para evitar la complegidad computacional, el set original se compone de 88 mil palabras.
numeropalabras = 10000
(X_train, y_train),(X_test,y_test) = imdb.load_data(num_words=numeropalabras)

## Exploración de datos

tamaño del dataset

In [3]:
print('set de entrenamiento:   ', X_train.shape)
print('target de entrenamiento:', y_train.shape)
print('set de prueba:   ', X_test.shape)
print('target de prueba:', y_test.shape)

set de entrenamiento:    (25000,)
target de entrenamiento: (25000,)
set de prueba:    (25000,)
target de prueba: (25000,)


revisar elemento en X_train

In [4]:
%pprint

Pretty printing has been turned OFF


In [5]:
X_train[150]

[1, 5, 14, 9, 6, 55, 1193, 22, 13, 203, 30, 355, 21, 14, 9, 4, 236, 22, 121, 13, 1192, 2967, 3622, 35, 779, 284, 37, 5844, 4, 217, 5, 2132, 6, 749, 10, 10, 2636, 4252, 5, 2931, 4517, 26, 82, 321, 36, 26, 2, 5, 4960, 2, 1786, 8, 358, 4, 704, 117, 122, 36, 124, 51, 62, 593, 375, 10, 10, 4, 1381, 5, 732, 26, 821, 5, 1249, 14, 16, 159, 4, 504, 7, 3728, 4913, 10, 10, 51, 9, 91, 1193, 44, 14, 22, 9, 4, 192, 15, 1370, 40, 14, 131, 1778, 11, 938, 704, 3834, 131, 2, 543, 84, 12, 9, 220, 6, 1117, 5, 6, 320, 237, 4, 3286, 325, 10, 10, 25, 80, 358, 14, 22, 12, 16, 814, 11, 4, 3968, 8084, 7, 1226, 7111, 63, 131, 1778, 43, 92, 1278, 501, 15, 8, 6353, 2, 15, 1609, 131, 47, 24, 77, 2, 237, 2, 2, 158, 158]

**Observacion**

Al explorar el elemento 150 encontramos una lista de números que nos índican cuál es el elemento que más se repite dentro del corpus de palabras. Por ejemplo, el 1 significa que es la palabra más repetida, el 5 la quinta palabra más utilizada, etc.

Hay un detalle con este dataset y es que la librería keras espera corpus de palabras completas, por ejemplo, keras espera un corpus de 100 palabras pero si solo le dimos 90 el resto de palabras para llegar a 100 las denominaría en ceros. Este fenomeno recibe el nombre de padding y para evitarlo keras desfasa cada 3 elementos las palabras, por lo tanto, si queremos ver la palabra más popular como se menciona en el párrafo anterior, entonces debemos búscar la palabra número 3 y no la 1 como se menciona.

In [6]:
# llamar a word_index
word_to_index = imdb.get_word_index()
# índice de la palabra bad
print('índice de la palabra bad:', word_to_index['bad'])

índice de la palabra bad: 75


La palabra **bad** es la número 75 más éxitoza o más repetitiva dentro de todas las reseñas que se muestran. Ahora lo que necesitamos es poder dar un número y que nos devuelva la palabra, para ello debemos cambiar la relación de palabras a números.

In [7]:
index_to_word={index:word for (word,index) in word_to_index.items()}

In [8]:
word_to_index['bad']

75

armar lista con las palabras más populares

In [9]:
[index_to_word[i] for i in range(1,51)]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i', 'this', 'that', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'on', 'not', 'you', 'are', 'his', 'have', 'he', 'be', 'one', 'all', 'at', 'by', 'an', 'they', 'who', 'so', 'from', 'like', 'her', 'or', 'just', 'about', "it's", 'out', 'has', 'if', 'some', 'there', 'what', 'good', 'more']

obtener el corpus del elemento 123 de la lista

In [10]:
" ".join([index_to_word.get(i-3,"?") for i in X_train[123]])

'? beautiful and touching movie rich colors great settings good acting and one of the most charming movies i have seen in a while i never saw such an interesting setting when i was in china my wife liked it so much she asked me to ? on and rate it so other would enjoy too'

según el texto, verificamos si es una relación positiva (1) o negativa (0)

In [11]:
y_train[123]

1

## Preprocesamiento de Datos
---

Todo el trabajo que realizamos anteriormente fue para poder comprender mejor de qué se tratan los elementos o reseñas del dataset, sin embargo, a keras le interesan más los números.

Definir palabras por reseña en 200

In [12]:
numpal=200
X_train = pad_sequences(
    X_train,
    maxlen=numpal
    ) #<-- truncamos las reseñas
X_train.shape # <-- nuevo split

(25000, 200)

In [13]:
X_test = pad_sequences(
    X_test,
    maxlen=numpal
    )
X_test.shape

(25000, 200)

Dividir los datos

In [14]:
X_test, X_val, y_test, y_val = train_test_split(
    X_test,
    y_test,
    random_state=11,
    test_size=0.20
    )

In [15]:
X_test.shape

(20000, 200)

In [16]:
X_val.shape

(5000, 200)

In [17]:
y_val.shape

(5000,)

## Crear Red Neuronal
---

In [18]:
rnn = Sequential() # <-- instanciar modelo
rnn.add(           # <-- capa embeding
        Embedding(
        input_dim=numeropalabras,
        output_dim=128,
        input_length=numpal
        )
    )
rnn.add( # <-- capa de recursión
    LSTM(
        units=128,
        dropout=0.2,
        recurrent_dropout=0.2
        )
    )
rnn.add( # <-- capa densa
    Dense(
        units=1,
        activation='sigmoid'
        )
    )
rnn.compile( # <-- compilar la red
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
    )

In [19]:
rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          1280000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1411713 (5.39 MB)
Trainable params: 1411713 (5.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Entrenar y evaluar modelo
---

Usar fit para entrenar modelo

In [20]:
rnn.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=(
        X_test,
        y_test
        )
    )

Epoch 1/10
782/782 [==============================] - 583s 741ms/step - loss: 0.4402 - accuracy: 0.7956 - val_loss: 0.3422 - val_accuracy: 0.8536
Epoch 2/10
782/782 [==============================] - 581s 744ms/step - loss: 0.2762 - accuracy: 0.8922 - val_loss: 0.3617 - val_accuracy: 0.8522
Epoch 3/10
782/782 [==============================] - 573s 733ms/step - loss: 0.2001 - accuracy: 0.9236 - val_loss: 0.3540 - val_accuracy: 0.8551
Epoch 4/10
782/782 [==============================] - 575s 736ms/step - loss: 0.1580 - accuracy: 0.9416 - val_loss: 0.3756 - val_accuracy: 0.8589
Epoch 5/10
782/782 [==============================] - 575s 735ms/step - loss: 0.1185 - accuracy: 0.9562 - val_loss: 0.4097 - val_accuracy: 0.8601
Epoch 6/10
782/782 [==============================] - 571s 730ms/step - loss: 0.0919 - accuracy: 0.9675 - val_loss: 0.4491 - val_accuracy: 0.8641
Epoch 7/10
782/782 [==============================] - 583s 746ms/step - loss: 0.0690 - accuracy: 0.9760 - val_loss: 0.5682 -

In [22]:
rnn.evaluate(X_val, y_val)

157/157 [==============================] - 16s 93ms/step - loss: 0.6959 - accuracy: 0.8526


[0.6959109902381897, 0.8525999784469604]

## Conclusiones

Después de completar con éxito cada uno de los pasos para desarrollar nuestro modelo de recursión, llegamos a los siguientes puntos:

- El modelo se desarrolló con un dataset propio de la librería de keras, el cual contiene hasta 80 mil reseñas de películas.
- Por temas de complejidad computacional, nuestro modelo se entrenó solo con 10 mil reseñas.
- El modelo LSTM se construyó con solo 3 capas debido a la complejidad computacional.
- El modelo logró una exactitud final de 85% mientras que se logró una función de pérdida de hasta un 69%.
- Lo anterior significa que, aunque el modelo cuenta con una buena precisión,
 aún no es capaz de distingir correctamente entre comentarios positivos o negativos que existen dentro de las reseñas.
- Algunas recomendaciones para mejorar el modelo incluyen:
  - Entrenar al modelo con más o menos datos.
  - Cambiar el rango de validación.
  - Aumentar capas de entrenamiento.

Por lo pronto nos quedaremos con este resultado, en otro proyecto estarémos utilizando esta red implementando mejoras para lograr mejores proyecciones en nuestro objetivo.